在训练神经网络时，最常用的算法是反向传播。在该算法中，根据损失函数相对于给定参数的梯度来调整参数（模型权重）。

为了计算这些梯度，PyTorch内置了一个名为torch.autograd的微分引擎。它支持自动计算任何计算图的梯度。

考虑最简单的单层神经网络，具有输入x、参数w和b以及一些损失函数。它可以通过以下方式在PyTorch中定义：

In [1]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
"""
在这个网络中，w和b是我们需要优化的参数。
因此，我们需要能够计算这些变量的损失函数梯度。
为此，我们设置了这些张量的requires_grad属性
您可以在创建张量时设置requires_grad的值
或者稍后使用x.requires_grad_（True）方法设置。
"""
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

我们应用于张量来构造计算图的函数实际上是function类的对象。

该对象知道如何计算正向函数，以及如何在反向传播步骤中计算其导数。

反向传播函数的引用存储在张量的grad_fn属性中。您可以在文档中找到函数的更多信息

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x00000238ED9F0190>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x00000238EFD80C70>


为了优化神经网络中参数的权重，我们需要计算损失函数相对于参数的导数。

使用loss.backward()计算反向传播的参数

然后通过w.grad b.grad

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3322, 0.0708, 0.3268],
        [0.3322, 0.0708, 0.3268],
        [0.3322, 0.0708, 0.3268],
        [0.3322, 0.0708, 0.3268],
        [0.3322, 0.0708, 0.3268]])
tensor([0.3322, 0.0708, 0.3268])


我们只能获得计算图的叶节点的grad属性，其中requires_grad属性设置为True。对于图中的所有其他节点，渐变将不可用。
出于性能原因，我们只能在给定的图形上使用反向一次来执行梯度计算。如果我们需要对同一个图执行多个反向调用，则需要将retain_graph=True传递给反向调用。

默认情况下，requires_grad=True的所有张量都在跟踪其计算历史并支持梯度计算。然而，在某些情况下，我们不需要这样做，例如，当我们训练了模型，只想将其应用于一些输入数据时，即我们只想通过网络进行前向计算。我们可以通过用torch.no_grad()块包围我们的计算代码来停止跟踪计算：

In [4]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


实现相同结果的另一种方法是在张量上使用detach()方法：

In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False
